In [1]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [2]:
import pickle
import codecs
import collections
import keras.preprocessing
from sklearn.model_selection import KFold
import csv

/Users/eqsk134/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense,GRU,Reshape,TimeDistributed,Bidirectional,Dropout,Masking
from keras_contrib.layers import CRF
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding,TimeDistributed,Flatten
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import GRU, Dropout
from keras.models import load_model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Masking
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
from keras import backend as K
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers import Embedding,TimeDistributed,Flatten
from keras.layers import BatchNormalization,Dropout


In [4]:
input_pbs = pd.read_csv('Thaipbs-tokenize_include_stop.csv')

In [5]:
input_pbs.head()

,Unnamed: 0,Unnamed: 0.1,headline,date,DOW,time,view,category,tag,hour,numTag,token,numToken,numChar
0,0,0,ฝุ่น PM2.5 : ทส.เตรียมตั้งศูนย์แก้ปัญหาหมอกควั...,05/04/2562,FRI,19:37,177,สิ่งแวดล้อม,"ฝุ่นเชียงใหม่,ฝุ่นPM2.5,ฝุ่นคลุมเมือง,เชียงใหม...",19,10,"ฝุ่น,PM,ทส.,เตรียม,ตั้ง,ศูนย์,แก้,ปัญหา,หมอก,ค...",12,47
1,1,1,"เลือกตั้ง 2562 : ""เพื่อไทย"" เตรียมยื่น กกต.จัด...",05/04/2562,FRI,19:20,702,การเมือง,"เลือกตั้ง62,เพื่อ่ไทย,กกต.,นับคะแนน,ไทยพีบีเอส...",19,6,"เลือกตั้ง,2562,เพื่อ,ไทย,เตรียม,ยื่น,กกต.,จัด,...",12,58
2,2,2,เตรียมเอาผิด รพ.เอกชน 58 แห่งไม่ส่งข้อมูลราคายา,05/04/2562,FRI,19:18,583,สาธารณสุข,"ยา,เวชภัณฑ์ฯ,โรงพยาบาล,ค่าบริการทางการแพทย์,กร...",19,9,"เตรียม,เอา,ผิด,รพ.เอกชน,แห่ง,ไม่,ส่ง,ข้อมูล,รา...",10,42
3,3,3,ตั้งข้อสังเกตปลดล็อกกัญชา เอื้อนายทุนหรือไม่?,05/04/2562,FRI,19:16,928,อาชญากรรม,"กัญชา,มูลนิธิข้าวขวัญ,ThaiPBSnews",19,3,"ตั้ง,ข้อ,สังเกต,ปลด,ล็อก,กัญชา,เอื้อ,นาย,ทุน,ห...",11,43
4,4,4,"คำพิพากษาเต็ม ""เปรมชัย"" หลุดคดีครอบครองซากเสือดำ",05/04/2562,FRI,19:29,5163,สิ่งแวดล้อม,"เสือดำ,เปรมชัย,เขตรักษาพันธุ์สัตว์ป่าทุ่งใหญ่น...",19,5,"คำ,พิพากษา,เต็ม,เปรมชัย,หลุดคดี,ครอบครอง,ซาก,เ...",8,44


In [6]:
# input_pbs.drop('Unnamed: 0.1', axis=1, inplace=True)
# input_pbs.drop('Unnamed: 1', axis=1, inplace=True)

In [7]:
input_pbs.head()

,Unnamed: 0,Unnamed: 0.1,headline,date,DOW,time,view,category,tag,hour,numTag,token,numToken,numChar
0,0,0,ฝุ่น PM2.5 : ทส.เตรียมตั้งศูนย์แก้ปัญหาหมอกควั...,05/04/2562,FRI,19:37,177,สิ่งแวดล้อม,"ฝุ่นเชียงใหม่,ฝุ่นPM2.5,ฝุ่นคลุมเมือง,เชียงใหม...",19,10,"ฝุ่น,PM,ทส.,เตรียม,ตั้ง,ศูนย์,แก้,ปัญหา,หมอก,ค...",12,47
1,1,1,"เลือกตั้ง 2562 : ""เพื่อไทย"" เตรียมยื่น กกต.จัด...",05/04/2562,FRI,19:20,702,การเมือง,"เลือกตั้ง62,เพื่อ่ไทย,กกต.,นับคะแนน,ไทยพีบีเอส...",19,6,"เลือกตั้ง,2562,เพื่อ,ไทย,เตรียม,ยื่น,กกต.,จัด,...",12,58
2,2,2,เตรียมเอาผิด รพ.เอกชน 58 แห่งไม่ส่งข้อมูลราคายา,05/04/2562,FRI,19:18,583,สาธารณสุข,"ยา,เวชภัณฑ์ฯ,โรงพยาบาล,ค่าบริการทางการแพทย์,กร...",19,9,"เตรียม,เอา,ผิด,รพ.เอกชน,แห่ง,ไม่,ส่ง,ข้อมูล,รา...",10,42
3,3,3,ตั้งข้อสังเกตปลดล็อกกัญชา เอื้อนายทุนหรือไม่?,05/04/2562,FRI,19:16,928,อาชญากรรม,"กัญชา,มูลนิธิข้าวขวัญ,ThaiPBSnews",19,3,"ตั้ง,ข้อ,สังเกต,ปลด,ล็อก,กัญชา,เอื้อ,นาย,ทุน,ห...",11,43
4,4,4,"คำพิพากษาเต็ม ""เปรมชัย"" หลุดคดีครอบครองซากเสือดำ",05/04/2562,FRI,19:29,5163,สิ่งแวดล้อม,"เสือดำ,เปรมชัย,เขตรักษาพันธุ์สัตว์ป่าทุ่งใหญ่น...",19,5,"คำ,พิพากษา,เต็ม,เปรมชัย,หลุดคดี,ครอบครอง,ซาก,เ...",8,44


In [8]:
IQRval = input_pbs['view'].describe().loc['75%']-input_pbs['view'].describe().loc['25%']

In [9]:
outlierMin = max(input_pbs['view'].describe().loc['25%']-1.5*IQRval,0)
outlierMax = input_pbs['view'].describe().loc['75%']+1.5*IQRval
print(outlierMin,outlierMax)

0 3499.5


In [10]:
input_pbs = input_pbs[(input_pbs['view']<=outlierMax) & (input_pbs['view']>=outlierMin)]

In [11]:
input_pbs['numToken'].describe()

count    24511.000000
mean        11.389662
std          3.354892
min          1.000000
25%          9.000000
50%         11.000000
75%         13.000000
max         28.000000
Name: numToken, dtype: float64

In [12]:
input_pbs['view'].describe()

count    24511.000000
mean       755.928603
std        754.956352
min         20.000000
25%        228.000000
50%        454.000000
75%       1006.000000
max       3496.000000
Name: view, dtype: float64

In [13]:
MAXVIEW = input_pbs['view'].max()
MAXVIEW

3496

## Prepare data for Keras

In [14]:
inputText = []
for i in range(27974):
    try:
        inputText.append(input_pbs.loc[:,'token'][i].split(','))
    except:
        x = 0

In [15]:
inputLabel = []
for view in input_pbs['view']:
    inputLabel.append(view/MAXVIEW)

In [16]:
print(len(inputText),len(inputLabel))

24511 24511


In [17]:
input_pbs.loc[:,'token']

0        ฝุ่น,PM,ทส.,เตรียม,ตั้ง,ศูนย์,แก้,ปัญหา,หมอก,ค...
1        เลือกตั้ง,2562,เพื่อ,ไทย,เตรียม,ยื่น,กกต.,จัด,...
2        เตรียม,เอา,ผิด,รพ.เอกชน,แห่ง,ไม่,ส่ง,ข้อมูล,รา...
3        ตั้ง,ข้อ,สังเกต,ปลด,ล็อก,กัญชา,เอื้อ,นาย,ทุน,ห...
6        ปิด,ล้อม,ตลาด,โรงเกลือ,รวบ,แรงงาน,ข้าม,แดน,ผิด...
7        เลือกตั้ง,2562,ลุ้น,พรรค,การ,เมือง,ได้,ส.ส.,ปา...
8        สงกรานต์,เดินทาง,ช่วง,สงกรานต์,ใช้,เส้นทาง,ไหน,ดี
9        เตือน,นัก,ท่องเที่ยว,คน,ไทย,ใน,เกาหลีใต้,ระวัง...
10       คน,เครียด,การ,เมือง,แห่,โทร,คุย,สาย,ด่วน,สุขภา...
12                 ด่วน,ไฟ,ไหม้,ลาน,จอด,รถ,ทบ.,เสียหาย,คัน
13                      ช้าง,รวม,โขลง,ออก,จาก,ป่าหาน้ำ,กิน
15       ฝุ่น,PM,ทดสอบ,“,เครื่อง,ดูด,ฝุ่น,PM,”,ของ,กลุ่...
16           ขนส่ง,ฯ,สั่ง,ปรับ,000,บาท,แท็กซี่,ดู,หนัง,โป๊
17       บาง,จาก,ฯ,ยืนยัน,ไม่,ปรับ,ขึ้น,ราคา,น้ำมัน,ช่ว...
18       เลือกตั้ง,2562,กกต.,เร่ง,ตรวจสอบ,เขต,คะแนน,มาก...
19       พระปฐมบรมราชโองการ,พระราชปณิธาน,ใน,พระราชพิธี,...
20       เช็ก,แล้ว,น้ำ,คูเมืองเชียงใหม่,ทะเลอันดามัน,สะ.

In [18]:
words = []
for sentence in inputText:
    for word in sentence:
        words.append(word)
        
word_count = list()
word_count.extend(collections.Counter(words).most_common(len(set(words))))
word_count = word_count[:len(word_count)//4]
word_count.append(("UNK",0))

train_word = set()
for i in word_count:
    train_word.add(i[0])

In [19]:
len(word_count)

4955

In [20]:
all_token = set()
for word in train_word:
    all_token.add(word)

In [21]:
all_token = list(all_token)
all_token.insert(0,'for padding')
all_token.insert(1,'UNK')

In [22]:
len(all_token)

4957

In [23]:
token_map = dict(zip(all_token, range(len(all_token))))
token_map_reverse = dict(zip(range(len(all_token)),all_token))

In [24]:
input_data = np.asarray(input_pbs['token'].str.split(','))

In [25]:
def count_word(sent):
    global all_token, token_map
    result = np.zeros(len(all_token))
    np_token = np.array(sent)
    str_token, str_token_count = np.unique(np_token, return_counts=True)
    for char, count in zip(str_token, str_token_count):
        if char not in token_map.keys():
            char = 'UNK'
        result[token_map[char]] = count
    return result

In [26]:
temp = np.vectorize(count_word, otypes=[object])(input_data)
x_f1 = np.array([[e for e in sl] for sl in temp.tolist()])

In [27]:
x_train=keras.preprocessing.sequence.pad_sequences(x_f1, maxlen=None, dtype='int32', padding='post', truncating='pre', value=0.)

In [28]:
hour_array = []
for hour in input_pbs['hour']:
    hour_array.append(hour)

In [29]:
DOW_array = []
all_DOW = []
for DOW in input_pbs['DOW']:
    DOW_array.append(DOW)
    all_DOW.append(DOW)

In [30]:
all_DOW = list(set(all_DOW))
map_DOW = {'FRI':5, 'WED':3, 'SAT':6, 'THU':4, 'MON':1, 'TUE':2, 'SUN':0}

In [31]:
category_array = []
all_category = []
for category in input_pbs['category']:
    category_array.append(category)
    all_category.append(category)

In [32]:
all_category = list(set(all_category))
map_category = {}
for i in range(len(all_category)):
    map_category[all_category[i]] = i

In [33]:
map_category

{'การศึกษา': 14,
 'การเมือง': 2,
 'กีฬา': 19,
 'จังหวัดชายแดนภาคใต้': 0,
 'ต่างประเทศ': 17,
 'บทความ-บทวิเคราะห์': 13,
 'ภัยพิบัติ': 10,
 'ภูมิภาค': 15,
 'รายงานพิเศษ': 12,
 'วิทยาศาสตร์เทคโนโลยี': 1,
 'ศิลปะ-บันเทิง': 8,
 'สตรีและเยาวชน': 6,
 'สังคม': 5,
 'สาธารณสุข': 4,
 'สิ่งแวดล้อม': 18,
 'อาชญากรรม': 7,
 'อาเซียน': 16,
 'เกษตร': 3,
 'เศรษฐกิจ': 11,
 'ไลฟ์สไตล์': 9}

In [34]:
map_DOW

{'FRI': 5, 'MON': 1, 'SAT': 6, 'SUN': 0, 'THU': 4, 'TUE': 2, 'WED': 3}

In [35]:
def onehot_label(hour):
    result = np.zeros(24)
    result[hour] = 1
    return result

In [36]:
def DOW_onehot_label(DOW):
    result = np.zeros(7)
    result[map_DOW[DOW]] = 1
    return result

In [37]:
def category_onehot_label(category):
    result = np.zeros(len(all_category))
    result[map_category[category]] = 1
    return result

In [38]:
x_train2 = np.zeros((x_train.shape[0],x_train.shape[1]+24+7+len(all_category)))
for i in range(x_train.shape[0]):
    temp = np.append(x_train[i],onehot_label(hour_array[i]))
    temp = np.append(temp,DOW_onehot_label(DOW_array[i]))
    x_train2[i] = np.append(temp,category_onehot_label(category_array[i]))

In [39]:
x_train.shape

(24511, 4957)

In [40]:
x_train = x_train2

In [41]:
x_train2 = np.zeros((x_train.shape[0],24+7+len(all_category)))
for i in range(x_train.shape[0]):
    temp = np.append([],onehot_label(hour_array[i]))
    temp = np.append(temp,DOW_onehot_label(DOW_array[i]))
    x_train2[i] = np.append(temp,category_onehot_label(category_array[i]))

In [42]:
x_train[0]

array([0., 0., 0., ..., 0., 1., 0.])

In [43]:
x_train2.shape

(24511, 51)

In [44]:
x_train, x_test =  train_test_split(x_train, test_size=0.2,random_state=1)

In [45]:
x_train2, x_test2 =  train_test_split(x_train2, test_size=0.2,random_state=1)

In [46]:
print("train size",x_train.shape)
print("test size",x_test.shape)
print("train size",x_train2.shape)
print("test size",x_test2.shape)

train size (19608, 5008)
test size (4903, 5008)
train size (19608, 51)
test size (4903, 51)


In [47]:
y_train = inputLabel
y_train = np.asarray(y_train).reshape(-1,1)
y_train2 = inputLabel
y_train2 = np.asarray(y_train2).reshape(-1,1)

In [48]:
y_train, y_test =  train_test_split(y_train, test_size=0.2,random_state=1)
y_train2, y_test2 =  train_test_split(y_train2, test_size=0.2,random_state=1)

In [49]:
print("train size",len(y_train))
print("test size",len(y_test))

train size 19608
test size 4903


## Create Model

In [50]:
def create_model():    
    input1 = Input(shape=(x_train.shape[1],))
    x = Dense(1024, activation='relu')(input1)
    x = Dropout(0.5)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(1,activation='linear')(x)
    model = Model(inputs=input1, outputs=x)
    adam  = Adam(lr=0.0001)
    model.compile(optimizer=adam,  loss='mean_absolute_percentage_error')
    
    return model


In [51]:
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5008)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              5129216   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
__________

In [52]:
weight_path='./model_eq_test1.h5'
callbacks_list = [
        ModelCheckpoint(
            weight_path,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
]

In [215]:
model.fit(x_train, y_train, batch_size=32 ,epochs=50, verbose=1, validation_split=0.2,callbacks=callbacks_list)

Train on 15686 samples, validate on 3922 samples
Epoch 1/50
 5408/15686 [=========>....................] - ETA: 49s - loss: 1340.6441

KeyboardInterrupt: 

In [53]:
def create_model2():    
    input1 = Input(shape=(x_train2.shape[1],))
    x = Dense(128, activation='relu')(input1)
    x = Dropout(0.4)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = BatchNormalization(momentum=0.99, epsilon=0.001)(x)
    x = Dense(1,activation='linear')(x)
    model = Model(inputs=input1, outputs=x)
    adam  = Adam(lr=0.0001)
    model.compile(optimizer=adam,  loss='mean_absolute_percentage_error')
    
    return model
model2 = create_model2()
model2.summary()
weight_path='./model_eq_test2.h5'
callbacks_list2 = [
        ModelCheckpoint(
            weight_path,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
]
model2.fit(x_train2, y_train2, batch_size=32 ,epochs=50, verbose=1, validation_split=0.2,callbacks=callbacks_list2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 51)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               6656      
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 128)               512       
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 256)               1024      
__________

15686/15686 [==============================] - 12s 741us/step - loss: 69.6221 - val_loss: 68.0212

Epoch 00035: val_loss did not improve
Epoch 36/50
15686/15686 [==============================] - 11s 719us/step - loss: 69.7386 - val_loss: 73.7041

Epoch 00036: val_loss did not improve
Epoch 37/50
15686/15686 [==============================] - 11s 698us/step - loss: 69.7280 - val_loss: 63.3317

Epoch 00037: val_loss did not improve
Epoch 38/50
15686/15686 [==============================] - 11s 709us/step - loss: 69.4588 - val_loss: 63.5668

Epoch 00038: val_loss did not improve
Epoch 39/50
15686/15686 [==============================] - 11s 706us/step - loss: 69.3948 - val_loss: 62.9214

Epoch 00039: val_loss did not improve
Epoch 40/50
15686/15686 [==============================] - 11s 732us/step - loss: 70.1314 - val_loss: 62.5674

Epoch 00040: val_loss did not improve
Epoch 41/50
15686/15686 [==============================] - 11s 730us/step - loss: 68.9324 - val_loss: 62.4739

Epoch 0

In [216]:
weight_model='model_eq_test1.h5'
model = create_model()
model.load_weights(weight_model)
model._make_predict_function()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 5008)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 1024)              5129216   
_________________________________________________________________
dropout_25 (Dropout)         (None, 1024)              0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_32 (Dense)             (None, 256)               262400    
_________________________________________________________________
dropout_26 (Dropout)         (None, 256)               0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 256)               1024      
__________

In [54]:
weight_model='model_eq_test2.h5'
mode2l = create_model2()
model2.load_weights(weight_model)
model2._make_predict_function()
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 51)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               6656      
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 128)               512       
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 256)               1024      
__________

## Evaluate Model

In [55]:
y_pred = model.predict(x_test)
y_pred2 = model2.predict(x_test2)

In [56]:
def evaluate(preds, testY):
    diff = preds.flatten() - testY
    percentDiff = (diff / testY) * 100
    absPercentDiff = np.abs(percentDiff)
    mean = np.mean(absPercentDiff)
    std = np.std(absPercentDiff)
    print(mean,std)

In [219]:
evaluate(y_pred,y_test)

76.27584938142674 85.86256582602603


In [57]:
evaluate(y_pred2,y_test2)

64.22962326435568 44.208005989386955


In [212]:
for i in range(50):
    print(y_test2[i],y_pred2[i])

[0.02974828] [0.0539628]
[0.027746] [0.08968236]
[0.02688787] [0.04178575]
[0.0471968] [0.05062674]
[0.20709382] [0.06351797]
[0.07265446] [0.06246524]
[0.42419908] [0.09298383]
[0.42620137] [0.08098318]
[0.10297483] [0.04866322]
[0.80234554] [0.06935862]
[0.88501144] [0.04340786]
[0.05263158] [0.04617205]
[0.06607551] [0.04584087]
[0.10411899] [0.06545265]
[0.11241419] [0.06261023]
[0.13701373] [0.06435726]
[0.06006865] [0.05427107]
[0.12099542] [0.04684357]
[0.36556064] [0.06256173]
[0.62328375] [0.07437061]
[0.03718535] [0.04877417]
[0.02088101] [0.10110772]
[0.10040046] [0.06178455]
[0.03632723] [0.04904272]
[0.51144165] [0.06245408]
[0.05577803] [0.04684357]
[0.04691076] [0.06323326]
[0.06035469] [0.05538625]
[0.28003432] [0.06748351]
[0.13443936] [0.04875656]
[0.06149886] [0.04487899]
[0.09982838] [0.0555429]
[0.22625858] [0.04334378]
[0.26315789] [0.05912165]
[0.35154462] [0.0892385]
[0.79405034] [0.04818447]
[0.5666476] [0.10698849]
[0.07065217] [0.0459474]
[0.24313501] [0.0663